In [81]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from IPython.display import display

import tensorflow as tf
from tensorflow.keras.layers import (Input, Dense, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D, Activation, BatchNormalization, Dropout)
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import (ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler, CSVLogger, LambdaCallback)
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from glob import glob
import cv2

# Additional imports for visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn

In [30]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [31]:
#Dataset Path
train_path = '/content/drive/MyDrive/Train'
test_path = '/content/drive/MyDrive/Test'

In [32]:
folders = glob ('/content/drive/MyDrive/Train/*')
print(len(folders))

2


In [ ]:
# Define the image size and number of classes
img_size = (256, 256)
num_classes = 2

In [33]:
# Create the model from the pre-trained model vgg16

model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
model = Sequential()

for layer in model_vgg16.layers:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False # Not trainable weights

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 64, 64, 256)      

In [ ]:
import glob
from tqdm import tqdm

img_size = 256
for i in os.listdir('/content/drive/MyDrive/Test'):
    files = glob.glob(pathname=str(os.path.join('/content/drive/MyDrive/Test', i, '*.jpg')))

    for j in tqdm(files):  # Use tqdm as a function
        # Process each JPEG file
        pass


100%|██████████| 12/12 [00:00<00:00, 213269.69it/s]


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [80]:
from keras.preprocessing.image import ImageDataGenerator

# Example configuration, you can modify this based on your requirements
data_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Now you can use data_datagen in your flow_from_directory method
train_set = data_datagen.flow_from_directory(
    '/content/drive/MyDrive/Train',
    target_size=(256, 256),
    batch_size=64,
    class_mode='binary',
)

Found 104 images belonging to 2 classes.


In [40]:
from keras.preprocessing.image import ImageDataGenerator

# Example configuration for test data augmentation (you can modify based on your needs)
test_datagen = ImageDataGenerator(rescale=1./255)

# Now you can use test_datagen in your flow_from_directory method
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Test',
    target_size=(256, 256),
    batch_size=64,
    class_mode='binary',
)

Found 24 images belonging to 2 classes.


In [42]:
import tqdm  # If you haven't imported tqdm already

batch_size = 64  # Adjust the batch size based on your needs

train_generator.reset()
X_train, y_train = next(train_generator)

for i in tqdm.tqdm(range(int(train_generator.n/batch_size)-1)):
    img, label = next(train_generator)
    X_train = np.append(X_train, img, axis=0)
    y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)
    # Your additional code inside the loop

0it [00:00, ?it/s]

(64, 256, 256, 3) (64,)


In [49]:
X_test, y_test = next(test_set)
for i in tqdm.tqdm(range(int(train_generator.n/batch_size)-1)):
  img, label = next(train_generator)
  X_train = np.append(X_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

0it [00:00, ?it/s]

(64, 256, 256, 3) (64,)


In [50]:
from keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator with desired augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to apply augmentation twice to each dataset
def augment_dataset(dataset):
    augmented_data = []
    for img in dataset:
        img = np.expand_dims(img, axis=0)  # Expanding dimensions for batch size
        for _ in range(1):  # Apply augmentation twice
            for batch in datagen.flow(img, batch_size=1):
                augmented_data.append(np.squeeze(batch))  # Remove the batch dimension
                break  # Stop the loop after one augmented image
    return np.array(augmented_data)

In [55]:
from keras.models import Model
import tensorflow as tf
from keras import backend as K
metrics = [
        'accuracy',
        tf.keras.metrics.AUC(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.F1Score(),
        tf.keras.metrics.SpecificityAtSensitivity(0.5),
        tf.keras.metrics.SensitivityAtSpecificity(0.5),
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.TruePositives(),]

In [56]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.1), loss = 'binary_crossentropy',metrics = metrics)

In [59]:
# Assuming y_train_combined is your binary labels
y_train_combined = y_train_combined.reshape((-1, 1))

In [60]:
from keras.utils import to_categorical

# Assuming num_classes is the number of classes in your problem
num_classes = 2
y_train_combined = to_categorical(y_train_combined, num_classes)

In [69]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
# Add your layers here

# For binary classification, use a single neuron with sigmoid activation
model.add(Dense(1, activation='sigmoid'))

# Compile the model with an appropriate optimizer, loss, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [71]:
y_train_combined = y_train_combined.ravel()  # or y_train_combined = y_train_combined.flatten()

In [72]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [74]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
# Add your layers here

# For binary classification, use a single neuron with sigmoid activation
model.add(Dense(1, activation='sigmoid'))

# Compile the model with an appropriate optimizer, loss, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [75]:
y_train_combined = y_train_combined.ravel()  # or y_train_combined = y_train_combined.flatten()

In [76]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [62]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
# Add your layers here

# For binary classification, use a single neuron with sigmoid activation
model.add(Dense(1, activation='sigmoid'))

# Compile the model with an appropriate optimizer, loss, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [63]:
y_train_combined = y_train_combined.reshape((-1, 1))

In [ ]:
import os
from glob import glob
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

train_path = '/content/drive/MyDrive/Train'

img_size = 256
for i in os.listdir(train_path):
    files = glob(pathname=str(os.path.join(train_path, i, '*.jpg')))

    inputs = []
    outputs = []

    for j in tqdm(files):
        input_img = Input(shape=(256,256, 3))
        x = Conv2D(16, (3, 3), activation='relu')(input_img)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(32, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Flatten()(x)
        x = Dense(64, activation='relu')(x)
        x = Dense(1, activation='sigmoid')(x)

        inputs.append(input_img)
        outputs.append(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

100%|██████████| 52/52 [00:06<00:00,  7.79it/s]


In [67]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
# Add your layers here

# For binary classification, use a single neuron with sigmoid activation
model.add(Dense(1, activation='sigmoid'))

# Compile the model with an appropriate optimizer, loss, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Extract the output layer from the functional model
output_layer = model_vgg16.output

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create your own model on top of the VGG16 base
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
predictions = layers.Dense(2, activation='softmax')(x)

model = keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Assuming your previous model definition using the functional API
# For example:
inputs = Input(shape=(256,256,3))
x = Dense(128, activation='relu')(inputs)
# ... add more layers as needed

# Output layer for binary classification with sigmoid activation
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Define your model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(256,256,3)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Now you can use model.fit_generator
model_history = model.fit_generator(
    train_set,
    validation_data=test_set,
    epochs=10
)



NameError: ignored

In [ ]:
# Fit Model
early_stopping_monitor = EarlyStopping(patience = 2)
model.fit(x_train,y_train,validation_split = 0.3,epochs = 100,batch_size = 64, callbacks = [early_stopping_monitor])

NameError: ignored

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output

<KerasTensor: shape=(None, 8, 8, 512) dtype=float32 (created by layer 'block5_pool')>